<a href="https://colab.research.google.com/github/jessica2907/Property-Data-Preprocessing/blob/main/Text_Blob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-dotenv

In [ ]:
import pandas as pd
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()

# Load OpenAI API key
client = OpenAI(api_key="INSERT KEY HERE")

# Function to get embedding
def get_embedding(text, model="text-embedding-ada-002"): #CHANGE THE MODEL TO A CHEAPER ONE!!
    text = text.replace("\n", " ").strip()
    return client.embeddings.create(input=[text], model=model).data[0].embedding

# Create description using all relevant (non-empty) columns
def create_natural_description(row, relevant_cols):
    parts = []
    for col in relevant_cols:
        value = row.get(col)
        if pd.notna(value) and str(value).strip() != "":
            parts.append(f"{col.replace('_', ' ')}: {str(value).strip()}")
    return ". ".join(parts)

# Process dataset and generate embeddings
def create_embeddings_of_dataset():
    df = pd.read_csv("/content/drive/MyDrive/Text_Blob/data_cleaned (1).csv")
    df.dropna(how='all', inplace=True)

    # Select all columns except "Propstack ID" and "Bilder"
    relevant_cols = [col for col in df.columns if col not in ["Propstack ID", "Bilder"]]

    # Generate text descriptions
    df['naturalDescription'] = df.apply(lambda row: create_natural_description(row, relevant_cols), axis=1)

    # Save only original columns and 'naturalDescription' column (no duplicate)
    columns_to_save = [col for col in df.columns if col != 'naturalDescriptionVector']  # Exclude 'naturalDescriptionVector'
    df_to_save = df[columns_to_save]  # All original columns and naturalDescription

    # Now append 'naturalDescription' at the far right
    df_to_save['naturalDescription'] = df['naturalDescription']

    df_to_save.to_csv('/content/drive/MyDrive/Text_Blob/data_cleaned_with_description.csv', index=False)

    print("✅ Embeddings saved to 'data_cleaned_with_description.csv'")
    print(df[['naturalDescription']].head(2))


def main():
    print("🔄 Generating descriptions from dataset...")
    create_embeddings_of_dataset()


if __name__ == "__main__":
    main()


🔄 Generating descriptions from dataset...
✅ Embeddings saved to 'data_cleaned_with_description.csv'
                                  naturalDescription
0  Kauf/Miete: Kauf. Kategorie: Wohnen. Typ: Haus...
1  Kauf/Miete: Kauf. Kategorie: Wohnen. Typ: Haus...
